<center> <h1 style="background-color:DarkSlateBlue; color:white" >Movie Recommendation System</h1> 

![Image](https://www.vshsolutions.com/wp-content/uploads/2020/02/recommender-system-for-movie-recommendation.jpg)

<center>
<br>    
<a id="top"></a>    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:DarkSlateBlue; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#Required libraries" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Required libraries<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">1</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#I/O" role="tab" aria-controls="profile" style="color:DarkSlateBlue">I/O<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">2</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Custom functions" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Custom functions<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">3</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Data loading" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Data loading<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">4</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Collaborative Filtering - User Based" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Collaborative Filtering - User Based<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">5</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Collaborative Filtering - Item Based" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Collaborative Filtering - Item Based<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">6</span></a>

<a id='Required libraries'></a>
<h1 style="color:DarkSlateBlue" >Required libraries</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [1]:
import sys, os
import re
path_entorno = sys.executable
if "env" in path_entorno:
    regexp = re.compile("envs(.*)$")
    entorno = regexp.search(path_entorno).group(1)
    print("Entorno:",entorno)
else:
    print("Entorno: base")
directorio = os.getcwd()
print("Directorio:",directorio)

Entorno: \env_recommender\python.exe
Directorio: C:\Users\fc00242\Desktop\Proyectos_Python\Mios\Github\Recommender Using Collaborative Filtering


In [2]:
#################Libraries##############
#General libraries
import os
import sys

#Data analysis libraries
import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np
import operator
import ast
import json
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)
from tqdm import tqdm_notebook

#Visualization libraries
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode (connected = True)
import ipywidgets as widgets

#Sklearn
from mlxtend.preprocessing import TransactionEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

#Nltk
import nltk
#nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#Image processing
from PIL import Image
import requests
from io import BytesIO

#String similarity
from pyjarowinkler import distance as jaro_distance
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from strsimpy.jaro_winkler import JaroWinkler

#Singular value decomposition
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

############################WARNINGS######################
import warnings
warnings.filterwarnings('ignore')

##################################DISPLAY###################################
from IPython.core.display import display, HTML,clear_output, Markdown
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:90% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

#from itertools import combinations,product
#!pip install strsimpy
#from strsimpy.normalized_levenshtein import NormalizedLevenshtein

<a id='I/O'></a>
<h1 style="color:DarkSlateBlue" >I/O</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

#### Input

In [3]:
path_metadata = './dat/movies_metadata.csv'
path_links_small = './dat/links_small.csv'
path_ratings_small = './dat/ratings_small.csv'

#### Output

In [23]:
path_ratings_complete = "./dat/ratings_complete.csv"
path_dict_similarity_collaborative_filtering = './dat/dict_similarity_collaborative_filtering.json'

<a id='Custom functions'></a>
<h1 style="color:DarkSlateBlue" >Custom functions</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [5]:
def calculate_jaro_distance(*, selected_title, all_possible_titles, num_similarities):
    #Jaro similarity
    similarity_jaro_list = []
    title_names = [title for title in all_possible_titles if str(title)!="nan"]
    for title_name in title_names:
        similarity_jaro = jaro_distance.get_jaro_distance(selected_title.lower(), title_name.lower())
        similarity_jaro_list.append(similarity_jaro)
    titles_dict_jaro = dict(zip(title_names, similarity_jaro_list))
    titles_dict_sorted_jaro = dict(sorted(titles_dict_jaro.items(), key=operator.itemgetter(1),reverse=True)[:num_similarities])
    df_similarities_jaro = pd.DataFrame(list(titles_dict_sorted_jaro.items()),columns = ['title','similarity']) 
    return df_similarities_jaro

In [21]:
def get_df_ratings_complete(df_ratings, df_links, df_metadata):
    # Merge three dataframes
    df_ratings_complete = df_ratings. \
        merge(df_links, how="inner"). \
        merge(df_metadata[['imdb_id', 'title']].drop_duplicates('title'), how="inner", left_on="imdbId", right_on="imdb_id")

    df_ratings_complete = df_ratings_complete[['userId', 'title', 'rating']]
    df_ratings_complete['userId'] = df_ratings_complete['userId'].astype(str)
    df_ratings_complete['rating'] = df_ratings_complete['rating'].astype(float)
    return df_ratings_complete

In [16]:
def get_most_likely_items_similarity_dict(*,items,max_number_of_predictions,dict_similarity):
    d = {}
    for movie in items:
        dict_movie = dict_similarity[movie]
        values = list(dict_movie.values())
        columns = list(dict_movie.keys())
        df_movie = pd.DataFrame(data = [values], columns=columns)
        d[movie] = df_movie
    df_similarity_filtered = pd.concat(d.values(), ignore_index=True).fillna(0)
    df_most_similar_items = df_similarity_filtered.sum(axis = 0).reset_index().rename(columns={'index':'title', 0:'similarity'}).sort_values(by="similarity",ascending=False)
    fig = px.bar(df_most_similar_items.head(max_number_of_predictions), x="title",y="similarity",title="Recommended movies (using cosine similarities)", 
           labels={'similarity': "Similarity"}, height=500)
    fig.show()

In [17]:
def get_most_likely_items_similarity_matrix(*,items,max_number_of_predictions,df_similarity):
    df_similarity_filtered = df_similarity[df_similarity['title'].isin(items)].drop(columns=items)
    display(df_similarity_filtered.head())
    df_most_similar_items = df_similarity_filtered.drop(columns=['title']).sum(axis = 0).reset_index().rename(columns={0:'similarity'}).sort_values(by="similarity",ascending=False)
    fig = px.bar(df_most_similar_items.head(max_number_of_predictions), x="title",y="similarity",title="Recommended movies (using cosine similarities)", 
           labels={'producto': "Similarity"}, height=500)
    fig.show()

<a id='Data loading'></a>
<h1 style="color:DarkSlateBlue" >Data loading</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [45]:
#################  1) METADATA DATASET     ##########################
df_metadata = pd.read_csv(path_metadata,low_memory=False)
df_metadata['id'] = df_metadata['id'].astype('str')
df_metadata['title'] = df_metadata['title'].str.title()
df_metadata['revenue'] = df_metadata['revenue'].astype('float')
df_metadata['imdb_id'] = df_metadata['imdb_id'].str.replace('tt','')
df_metadata = df_metadata[df_metadata['revenue']>10_000_000].drop_duplicates('title')

################   2) LINKS DATASET         #####################
df_links = pd.read_csv(path_links_small,low_memory=False,dtype=str)
display(df_links.head())

################   3) RATINGS SMALL DATASET ####################
df_ratings = pd.read_csv(path_ratings_small,dtype=str)
display(df_ratings.head())


##########################COLLABORATIVE FILTERING##############
df_ratings_complete = get_df_ratings_complete(df_ratings, df_links, df_metadata)
df_ratings_complete.to_csv(path_ratings_complete, sep=",",index=False)


#Some information about our data
number_of_users = df_ratings_complete[['userId']].drop_duplicates().shape[0]
number_of_movies = df_ratings_complete[['title']].drop_duplicates().shape[0]
number_of_ratings = df_ratings_complete.shape[0]
number_of_ratings_per_user = round(df_ratings_complete.shape[0]/number_of_users)
print("There are {} different users, {} different movies and {} ratings in total.".format(number_of_users,number_of_movies,number_of_ratings))
print("On average every user has rated {} movies".format(number_of_ratings_per_user))

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844
2,3,0113228,15602
3,4,0114885,31357
4,5,0113041,11862


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


There are 671 different users, 3422 different movies and 74429 ratings in total.
On average every user has rated 111 movies


In [46]:
#Image that we will use in the notebook
#response = requests.get("https://wpamelia.com/wp-content/uploads/2019/06/loading1.jpg")
#image = Image.open(BytesIO(response.content)) 

<a id='Collaborative Filtering - User Based'></a>
<h1 style="color:DarkSlateBlue"> Collaborative Filtering - User Based </h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [26]:
#Let's make the ratings dataset a surprise dataset
reader = Reader()
data = Dataset.load_from_df(df_ratings_complete[['userId', 'title', 'rating']], reader)

# Run 5-fold cross-validation and then print results
svd = SVD(random_state=111)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

#Let's train our model
trainset = data.build_full_trainset()
svd.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8749  0.8797  0.8764  0.8783  0.8869  0.8792  0.0041  
MAE (testset)     0.6716  0.6755  0.6721  0.6752  0.6812  0.6751  0.0034  
Fit time          3.20    3.26    3.18    3.35    3.40    3.28    0.09    
Test time         0.09    0.08    0.08    0.79    0.08    0.23    0.28    


In [27]:
############################RATINGS############################
fig = px.histogram(df_ratings_complete,x="rating",title="Distribution of ratings")
fig.update_layout(autosize=False,width=800,height=400).show()


#############################USERS#############################    
#Users that have rated many movies
active_users = df_ratings_complete.groupby("userId").agg({'rating':'count'}).reset_index().\
                rename(columns={'rating':'number_of_ratings'}).sort_values(by="number_of_ratings",ascending=False)
active_users['userId'] = 'ID' + active_users['userId']

#Figure                                                                          
fig = px.bar(active_users.head(20),
            x="userId",y="number_of_ratings",
            title="Users with highest number of ratings")
fig.update_layout(autosize=False,width=800,height=400).show()

                                                                           
##############################MOVIES###############################
#Popular movies
popular_movies = df_ratings_complete.groupby("title").agg({'userId':'count','rating':'mean'}).reset_index().\
                rename(columns={'userId':'number_of_ratings','rating':'average_rating'}).sort_values(by="number_of_ratings",ascending=False)
                                                                           
#Figure
fig = px.bar(popular_movies.head(20),
             x="title",y="number_of_ratings",#color="title",
             title="Movies with highest number of ratings")
fig.update_layout(autosize=False,width=800,height=400).show()

Our main goal would be to predict what are the top recommendations for different user according to their personal taste.

In [28]:
#Let's predict
@widgets.interact
def get_recommendations(user = '1'):
    users = df_ratings_complete['userId'].unique()
    if user in users:
        #Barchart of favourite movies by user
        movies_rated_by_user = df_ratings_complete[df_ratings_complete['userId'] == user].sort_values(by=["userId","rating"],ascending=False)
        movies_list = movies_rated_by_user['title'].tolist()
        #print("Number of ratings by user {}:".format(user),movies_rated_by_user.shape[0], "// Average rating by user {}:".format(user),movies_rated_by_user['rating'].mean())
        top_movies_rated_by_user = movies_rated_by_user[movies_rated_by_user['rating']>=3]
        fig1 = px.bar(top_movies_rated_by_user.head(10),x="title",y="rating",
                       color="title",
                       color_discrete_sequence = px.colors.qualitative.Vivid,
                       title="Highest rated movies by user {}".format(user))
        #fig1.update_layout(autosize=False,width=800,height=350).show()

        #Recommendations
        title_names = df_ratings_complete['title'].unique()
        predicted_ratings = []
        for title in title_names:
            predicted_rating = svd.predict(user, title)
            predicted_ratings.append(predicted_rating.est)
        titles_dict = dict(zip(title_names, predicted_ratings))
        titles_dict_sorted = dict(sorted(titles_dict.items(), key=operator.itemgetter(1),reverse=True))
        df_recommended_movies = pd.DataFrame(list(titles_dict_sorted.items()),columns = ['title','predicted_ratings'])
        
        df_recommended_movies = df_recommended_movies[~df_recommended_movies['title'].isin(movies_list)].head(10)

        #Barchart of the recommendations
        fig2 = px.bar(df_recommended_movies,x="title",y="predicted_ratings",
                       color="title",
                       color_discrete_sequence = px.colors.qualitative.Dark24,
                       title="Recommended movies for user {}".format(user))
        #fig2.update_layout(autosize=False,width=800,height=400).show()


        #Figure
        fig = make_subplots(rows=1, cols=2,subplot_titles=("Highest rated movies by user {} - movies already watched".format(user),
                                                           "Recommended movies for user {}".format(user)))


        for trace in fig1.data:
            fig.add_trace(trace, 1, 1)
        for trace in fig2.data:
            fig.add_trace(trace, 1, 2)

        fig.update_layout(barmode="stack")
        fig.update_layout(autosize=False,width=1400,height=600,showlegend=False).show()
    else:
        print("Not found")
        #display(image)

interactive(children=(Text(value='1', description='user'), Output()), _dom_classes=('widget-interact',))

There is a **big problem** with all the **user based techniques**. If we want to recommend a movie to a new user, we can't do it without retraining the model with the new user which is really computationally expensive. That's why we will see how to implement an item based collaborative filtering, where we can have an item similarity matrix pre-computed, and we just need to recommend movies that are similar to the ones the user has liked.

<a id='Collaborative Filtering - Item Based'></a>
<h1 style="color:DarkSlateBlue"> Collaborative Filtering - Item Based </h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [68]:
range(1,num_notas+1)

range(1, 4)

In [89]:
num_notas = 3
ratings_per_movie = df_ratings_complete.groupby('title').agg({'userId':'count'}).reset_index().sort_values(by="userId",ascending=False).rename(columns={'userId':'number_of_ratings'})
ratings_per_movie['quality'] = ratings_per_movie['number_of_ratings'].transform(lambda x: pd.qcut(x,num_notas, duplicates='drop',labels=['Low quality','Average quality','High quality']))
ratings_per_movie[ratings_per_movie['quality']=="Average quality"].head(100)

,title,number_of_ratings,quality
1990,Red,18,Average quality
2255,Sommersby,18,Average quality
2546,The Constant Gardener,18,Average quality
353,Bicentennial Man,18,Average quality
845,Escape To Witch Mountain,18,Average quality
2918,The Phantom,18,Average quality
1089,Guess Who'S Coming To Dinner,18,Average quality
1669,Mr. Mom,18,Average quality
954,For Your Eyes Only,18,Average quality
2836,The Man With The Golden Gun,18,Average quality


In [86]:
movie = "Forrest Gump"
print("Quality of prediction:",ratings_per_movie[ratings_per_movie['title']==movie]['quality'].values[0])

Quality of prediction: High quality


In [56]:
display(df_ratings_complete[df_ratings_complete['title'] == "The Great Gatsby"])
display(df_ratings_complete[df_ratings_complete['title'] == "Black Widow"])
display(df_ratings_complete[df_ratings_complete['title'] == "Hollywood Homicide"])
display(df_ratings_complete[df_ratings_complete['title'] == "Avatar"])

,userId,title,rating
73639,311,The Great Gatsby,3.5


,userId,title,rating
73641,311,Black Widow,2.5


,userId,title,rating
73622,311,Hollywood Homicide,3.5


,userId,title,rating
56839,15,Avatar,4.0
56840,26,Avatar,3.5
56841,31,Avatar,4.0
56842,48,Avatar,4.5
56843,72,Avatar,2.0
56844,73,Avatar,4.0
56845,78,Avatar,5.0
56846,81,Avatar,3.0
56847,84,Avatar,4.0
56848,93,Avatar,4.0


In [55]:
# Dataframe where every row is an user, the columns are all the possible movies and the values are the ratings (NaN if the user has not watched the movie)
pivot_ratings = df_ratings_complete.pivot(index='userId', columns='title', values='rating').fillna(0) #pivot_ratings = pd.pivot_table(df_ratings_complete,index='userId', columns='title', values='rating')
display(pivot_ratings[pivot_ratings['Avatar']!=0][['Avatar']])

# Cosine similarity matrix
similarity = cosine_similarity(pivot_ratings.T)##The distance is a positive measure, and we are calculating the similarity as 1-distance
df_similarity = pd.DataFrame(similarity, index=pivot_ratings.columns, columns=pivot_ratings.columns).reset_index()
display(df_similarity.head())
print("Similarity matrix shape:",df_similarity.shape)
print("Size of matrix:",sys.getsizeof(df_similarity)/10**6 ,"MB")

title,Avatar
userId,
104,4.5
149,3.5
15,4.0
152,4.5
157,3.5
186,5.0
199,3.0
205,3.0
212,4.0


title,title,(500) Days Of Summer,...And Justice For All,10,10 Cloverfield Lane,10 Things I Hate About You,"10,000 Bc",101 Dalmatians,102 Dalmatians,12 Years A Slave,127 Hours,13 Assassins,13 Going On 30,1408,15 Minutes,16 Blocks,17 Again,1941,2 Fast 2 Furious,2 Guns,"20,000 Leagues Under The Sea",2001: A Space Odyssey,2010,2012,2046,21,21 Grams,21 Jump Street,22 Jump Street,25Th Hour,27 Dresses,28 Days Later,28 Weeks Later,3 Idiots,30 Days Of Night,30 Minutes Or Less,300,3000 Miles To Graceland,300: Rise Of An Empire,3:10 To Yuma,42,47 Ronin,48 Hrs.,50 First Dates,50/50,54,8 Mile,88 Minutes,8Mm,9,...,Witness,Wolf Creek,Woodstock,Working Girl,World Trade Center,World War Z,Wrath Of The Titans,Wreck-It Ralph,Wrong Turn,Wyatt Earp,X-Men,X-Men Origins: Wolverine,X-Men: Apocalypse,X-Men: Days Of Future Past,X-Men: First Class,X-Men: The Last Stand,X2,Xanadu,Xxx,Xxx: State Of The Union,Y Tu Mamá También,Year One,Yeh Jawaani Hai Deewani,Yentl,Yes Man,Yogi Bear,You Again,You Don'T Mess With The Zohan,You Only Live Twice,You'Re Next,You'Ve Got Mail,"You, Me And Dupree",Young Adult,Young Doctors In Love,Young Frankenstein,Young Guns,Your Highness,Yu-Gi-Oh! The Movie,Zack And Miri Make A Porno,Zathura: A Space Adventure,Zelig,Zero Dark Thirty,Zodiac,Zombieland,Zoolander,Zoolander 2,Zoom,Zootopia,[Rec],Æon Flux
0,(500) Days Of Summer,1.000000,0.058854,0.113989,0.196231,0.283897,0.121806,0.129939,0.094110,0.146037,0.407882,0.202864,0.210097,0.062929,0.026903,0.289128,0.346245,0.035610,0.225193,0.068092,0.025307,0.218816,0.021684,0.237239,0.076462,0.202143,0.281115,0.299737,0.113106,0.196543,0.186308,0.233817,0.324802,0.139138,0.040164,0.156906,0.385433,0.036076,0.280586,0.267168,0.162512,0.151925,0.093013,0.213097,0.329913,0.025116,0.250343,0.000000,0.055626,0.332005,...,0.077050,0.137808,0.113945,0.033270,0.200148,0.346634,0.133987,0.288940,0.076012,0.016380,0.255550,0.205642,0.192812,0.231175,0.399936,0.144266,0.161570,0.000000,0.144961,0.134815,0.149668,0.182710,0.0,0.075104,0.274430,0.096296,0.173334,0.159436,0.023056,0.057778,0.156814,0.122565,0.192593,0.000000,0.077329,0.027347,0.196407,0.173334,0.199297,0.121644,0.099381,0.362845,0.309247,0.385191,0.213444,0.00000,0.000000,0.148747,0.127878,0.071454
1,...And Justice For All,0.058854,1.000000,0.018557,0.028964,0.075105,0.000000,0.166044,0.020732,0.000000,0.091338,0.000000,0.038352,0.049257,0.218395,0.046386,0.033475,0.000000,0.066477,0.000000,0.449793,0.263861,0.301382,0.025843,0.000000,0.025795,0.037652,0.018826,0.000000,0.068982,0.024456,0.055910,0.059520,0.000000,0.032934,0.000000,0.030341,0.000000,0.000000,0.031297,0.052398,0.000000,0.126183,0.022792,0.018093,0.244934,0.013072,0.000000,0.487495,0.000000,...,0.240396,0.000000,0.150730,0.379994,0.123091,0.000000,0.000000,0.000000,0.000000,0.120883,0.038472,0.000000,0.000000,0.000000,0.029814,0.026729,0.039957,0.169504,0.008105,0.000000,0.067972,0.000000,0.0,0.039590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.247799,0.000000,0.000000,0.438763,0.268500,0.083104,0.027875,0.000000,0.000000,0.000000,0.112835,0.025584,0.057854,0.037282,0.067422,0.00000,0.000000,0.000000,0.000000,0.000000
2,10,0.113989,0.018557,1.000000,0.106420,0.019031,0.000000,0.063979,0.000000,0.000000,0.058365,0.330748,0.000000,0.000000,0.170214,0.000000,0.000000,0.225309,0.000000,0.000000,0.228740,0.138776,0.000000,0.083083,0.000000,0.147432,0.041503,0.250746,0.220193,0.203222,0.029952,0.032681,0.079524,0.000000,0.000000,0.195424,0.092901,0.228254,0.187530,0.155718,0.591831,0.259794,0.479080,0.000000,0.066477,0.048646,0.048030,0.000000,0.000000,0.000000,...,0.165613,0.000000,0.145741,0.252984,0.351763,0.106465,0.578006,0.135274,0.000000,0.103635,0.058636,0.117429,0.000000,0.097079,0.113599,0.133670,0.072117,0.000000,0.046322,0.000000,0.049949,0.000000,0.0,0.145464,0.179267,0.609272,0.000000,0.144108,0.257019,0.000000,0.147644,0.000000,0.000000,0.000000,0.087368,0.101781,0.358464,0.000000,0.227884,0.000000,0.103646,0.288969,0.195466,0.140786,0.123862,0.0

Similarity matrix shape: (3422, 3423)
Size of matrix: 93.929367 MB


In [14]:
%%time
dict_similarity = {}
number_of_similarities_per_title = 50
for title in tqdm_notebook(df_similarity['title'].unique()):
    dict_similarity[title] = dict(df_similarity[df_similarity['title']==title].iloc[0].iloc[1:])
    dict_similarity[title] = dict(sorted(dict_similarity[title].items(), key=operator.itemgetter(1),reverse=True)[1:number_of_similarities_per_title])
print("Size of matrix:",sys.getsizeof(dict_similarity)/10**6 ,"MB")

#Saving the dictionary as a json file
with open(path_dict_similarity_collaborative_filtering, 'w') as fp:
    json.dump(dict_similarity, fp)

  0%|          | 0/3422 [00:00<?, ?it/s]

Size of matrix: 0.14756 MB
Wall time: 1min 6s


In [31]:
#Let's plot the results
# Button, text box, output
layout = widgets.Layout(width='400px', height='25px') #set width and height
butt_collaborative_filtering = widgets.Button(description='Display similar movies based on collaborative filtering',layout = layout,button_style='success')
items_collaborative_filtering = widgets.Text(value = 'The Dark Knight',description='Titles',layout=layout)
num_similar_items_collaborative_filtering = widgets.Dropdown(options=list(range(1,21)),value=10,description='Num similar items',disabled=False,layout=layout)
output_collaborative_filtering = widgets.Output()

def on_butt_clicked(b):
    with output_collaborative_filtering:
        clear_output()
        itemset = list(items_collaborative_filtering.value.split(","))
        itemset = [item.strip().title() for item in itemset]
        #print(itemset)
        items_no_reconocidos = [item for item in itemset if item not in list(dict_similarity.keys())]
        if len(items_no_reconocidos)==0:
            #################################ITEM-TO-ITEM COLLABORATIVE FILTERING USING COSINE SIMILARITIES##########################
            #get_most_likely_items_similarity_matrix(items = itemset, max_number_of_predictions = num_similar_items_collaborative_filtering.value, df_similarity = df_similarity)
            get_most_likely_items_cosine_similarity_dict(items = itemset, max_number_of_predictions = num_similar_items_collaborative_filtering.value, dict_similarity = dict_similarity)
        else:
            if len(items_no_reconocidos)==1:
                print(f"The title {items_no_reconocidos} is not recognised", "\n")
            if len(items_no_reconocidos)>1:
                print(f"The titles {items_no_reconocidos} are not recognised", "\n")
            for item in items_no_reconocidos:
                df_similarities_jaro = calculate_jaro_distance(selected_title = item, all_possible_titles = df_ratings_complete['title'].unique(), num_similarities = 1)
                print("Maybe you meant:",df_similarities_jaro['title'].values[0])
            
        
butt_collaborative_filtering.on_click(on_butt_clicked)
widgets.VBox([butt_collaborative_filtering,
              items_collaborative_filtering,
              num_similar_items_collaborative_filtering,
              output_collaborative_filtering])  